In [28]:
# 08-deployment.ipynb
# Cell 1: Setup and create FastAPI app

from google.cloud import storage
import os

PROJECT_ID = "openbook-ml-demo"
BUCKET_NAME = "openbook-data-lake"

client = storage.Client(project=PROJECT_ID)
bucket = client.bucket(BUCKET_NAME)

# Clean deploy folder
os.makedirs("deploy", exist_ok=True)

# FastAPI code - using REST API for Gemini (no SDK import issues)
fastapi_code = '''from fastapi import FastAPI
from pydantic import BaseModel
from typing import List
import httpx
import google.auth
from google.auth.transport.requests import Request

app = FastAPI(title="Openbook Copay Prediction API")

PROJECT_ID = "openbook-ml-demo"
LOCATION = "us-central1"
ENDPOINT_ID = "2273604228775673856"

class ProcedureInput(BaseModel):
    procedure_code: str
    procedure_cost: float

class PatientInput(BaseModel):
    patient_name: str
    insurance_carrier: str
    plan_type: str
    annual_maximum: float
    remaining_maximum: float
    deductible_remaining: float
    months_enrolled: int
    is_in_network: bool
    procedures: List[ProcedureInput]

class PredictionOutput(BaseModel):
    procedure_code: str
    procedure_cost: float
    predicted_copay: float
    predicted_insurance: float

class TreatmentPlanOutput(BaseModel):
    patient_name: str
    predictions: List[PredictionOutput]
    total_cost: float
    total_copay: float
    total_insurance: float
    treatment_plan_letter: str

@app.get("/health")
def health_check():
    return {"status": "healthy"}

@app.post("/predict", response_model=List[PredictionOutput])
def predict_copay(patient: PatientInput):
    predictions = []
    for proc in patient.procedures:
        coverage = 0.6 if patient.plan_type == "PPO" else 0.5
        copay = proc.procedure_cost * (1 - coverage)
        insurance = proc.procedure_cost - copay
        predictions.append(PredictionOutput(
            procedure_code=proc.procedure_code,
            procedure_cost=proc.procedure_cost,
            predicted_copay=round(copay, 2),
            predicted_insurance=round(insurance, 2)
        ))
    return predictions

@app.post("/treatment-plan", response_model=TreatmentPlanOutput)
def generate_treatment_plan(patient: PatientInput):
    predictions = predict_copay(patient)
    total_cost = sum(p.procedure_cost for p in predictions)
    total_copay = sum(p.predicted_copay for p in predictions)
    total_insurance = sum(p.predicted_insurance for p in predictions)
    
    gemini_input = f"""Patient: {patient.patient_name}
Insurance: {patient.insurance_carrier} {patient.plan_type}
Remaining Annual Maximum: ${patient.remaining_maximum}

Recommended Procedures:
"""
    for p in predictions:
        gemini_input += f"- {p.procedure_code}: ${p.procedure_cost} (You pay: ${p.predicted_copay}, Insurance pays: ${p.predicted_insurance})\\n"
    
    gemini_input += f"""
Total Cost: ${total_cost}
Your Estimated Cost: ${total_copay}
Insurance Pays: ${total_insurance}"""

    try:
        credentials, _ = google.auth.default()
        credentials.refresh(Request())
        url = f"https://{LOCATION}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{ENDPOINT_ID}:generateContent"
        response = httpx.post(
            url,
            headers={"Authorization": f"Bearer {credentials.token}"},
            json={"contents": [{"role": "user", "parts": [{"text": gemini_input}]}]},
            timeout=30.0
        )
        result = response.json()
        treatment_letter = result.get("candidates", [{}])[0].get("content", {}).get("parts", [{}])[0].get("text", "Error generating plan")
    except Exception as e:
        treatment_letter = f"Dear {patient.patient_name},\\n\\nTreatment plan unavailable. Total: ${total_cost}, Your cost: ${total_copay}\\n\\nAvalon Dental Team"

    return TreatmentPlanOutput(
        patient_name=patient.patient_name,
        predictions=predictions,
        total_cost=round(total_cost, 2),
        total_copay=round(total_copay, 2),
        total_insurance=round(total_insurance, 2),
        treatment_plan_letter=treatment_letter
    )
'''

with open("deploy/main.py", "w") as f:
    f.write(fastapi_code)

# Verify
print("First 5 lines of deploy/main.py:")
with open("deploy/main.py", "r") as f:
    for i, line in enumerate(f):
        if i < 5:
            print(line.rstrip())

print("\n✓ FastAPI app created")

First 5 lines of deploy/main.py:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List
import httpx
import google.auth

✓ FastAPI app created


In [29]:
# Cell 2: Create Dockerfile, requirements, cloudbuild, and Terraform

dockerfile = '''FROM python:3.10-slim

WORKDIR /app

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY main.py .

CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8080"]
'''

requirements = '''fastapi==0.104.1
uvicorn==0.24.0
pydantic==2.5.0
httpx==0.25.0
google-auth==2.23.0
'''

cloudbuild_yaml = '''steps:
  - name: 'gcr.io/cloud-builders/docker'
    args: ['build', '-t', 'gcr.io/$PROJECT_ID/openbook-api', '.']
  
  - name: 'gcr.io/cloud-builders/docker'
    args: ['push', 'gcr.io/$PROJECT_ID/openbook-api']
  
  - name: 'gcr.io/google.com/cloudsdktool/cloud-sdk'
    entrypoint: gcloud
    args:
      - 'run'
      - 'deploy'
      - 'openbook-api'
      - '--image'
      - 'gcr.io/$PROJECT_ID/openbook-api'
      - '--region'
      - 'us-central1'
      - '--platform'
      - 'managed'
      - '--allow-unauthenticated'

images:
  - 'gcr.io/$PROJECT_ID/openbook-api'
'''

terraform_main = '''terraform {
  required_providers {
    google = {
      source  = "hashicorp/google"
      version = "~> 5.0"
    }
  }
}

provider "google" {
  project = var.project_id
  region  = var.region
}

resource "google_cloud_run_service" "openbook_api" {
  name     = "openbook-api"
  location = var.region

  template {
    spec {
      containers {
        image = "gcr.io/${var.project_id}/openbook-api"
        resources {
          limits = {
            memory = "1Gi"
            cpu    = "1"
          }
        }
        env {
          name  = "PROJECT_ID"
          value = var.project_id
        }
      }
    }
  }

  traffic {
    percent         = 100
    latest_revision = true
  }
}

resource "google_cloud_run_service_iam_member" "public" {
  service  = google_cloud_run_service.openbook_api.name
  location = google_cloud_run_service.openbook_api.location
  role     = "roles/run.invoker"
  member   = "allUsers"
}

resource "google_pubsub_topic" "retrain_trigger" {
  name = "model-retrain-trigger"
}

resource "google_storage_bucket" "data_lake" {
  name     = "${var.project_id}-data-lake"
  location = var.region
}

output "api_url" {
  value = google_cloud_run_service.openbook_api.status[0].url
}
'''

terraform_vars = '''variable "project_id" {
  description = "GCP Project ID"
  default     = "openbook-ml-demo"
}

variable "region" {
  description = "GCP Region"
  default     = "us-central1"
}
'''

# Write all files
with open("deploy/Dockerfile", "w") as f:
    f.write(dockerfile)

with open("deploy/requirements.txt", "w") as f:
    f.write(requirements)

with open("deploy/cloudbuild.yaml", "w") as f:
    f.write(cloudbuild_yaml)

with open("deploy/main.tf", "w") as f:
    f.write(terraform_main)

with open("deploy/variables.tf", "w") as f:
    f.write(terraform_vars)

# Upload to GCS
for filename in ["main.py", "Dockerfile", "requirements.txt", "cloudbuild.yaml", "main.tf", "variables.tf"]:
    blob = bucket.blob(f"deployment/{filename}")
    blob.upload_from_filename(f"deploy/{filename}")

print("✓ All deployment files created:")
print("  - main.py (FastAPI)")
print("  - Dockerfile")
print("  - requirements.txt")
print("  - cloudbuild.yaml (CI/CD)")
print("  - main.tf (Terraform)")
print("  - variables.tf (Terraform)")
print("\n✓ Uploaded to gs://openbook-data-lake/deployment/")

✓ All deployment files created:
  - main.py (FastAPI)
  - Dockerfile
  - requirements.txt
  - cloudbuild.yaml (CI/CD)
  - main.tf (Terraform)
  - variables.tf (Terraform)

✓ Uploaded to gs://openbook-data-lake/deployment/


In [30]:
# Cell 3: Verify and deploy

# Verify main.py has correct imports (no vertexai)
print("Checking deploy/main.py imports:")
!head -10 deploy/main.py

print("\n" + "="*50)
print("If imports look correct (httpx, google.auth), run next cell to deploy")

Checking deploy/main.py imports:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List
import httpx
import google.auth
from google.auth.transport.requests import Request

app = FastAPI(title="Openbook Copay Prediction API")

PROJECT_ID = "openbook-ml-demo"

If imports look correct (httpx, google.auth), run next cell to deploy


In [32]:
requirements = '''fastapi==0.104.1
uvicorn==0.24.0
pydantic==2.5.0
httpx==0.25.0
google-auth==2.23.0
requests==2.31.0
'''

with open("deploy/requirements.txt", "w") as f:
    f.write(requirements)

print("✓ Added requests to requirements.txt")
!cat deploy/requirements.txt

✓ Added requests to requirements.txt
fastapi==0.104.1
uvicorn==0.24.0
pydantic==2.5.0
httpx==0.25.0
google-auth==2.23.0
requests==2.31.0


In [33]:
# Cell 4: Build and deploy to Cloud Run

!cd deploy && gcloud builds submit --tag gcr.io/openbook-ml-demo/openbook-api .

!gcloud run deploy openbook-api \
    --image gcr.io/openbook-ml-demo/openbook-api \
    --region us-central1 \
    --platform managed \
    --allow-unauthenticated

Creating temporary archive of 6 file(s) totalling 5.8 KiB before compression.
Uploading tarball of [.] to [gs://openbook-ml-demo_cloudbuild/source/1765899601.78179-afe9226a63834c1ab70ab82a0b1b2ee7.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/openbook-ml-demo/locations/global/builds/0fd2ce56-80b5-4d95-8ab8-976cbb95a245].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/0fd2ce56-80b5-4d95-8ab8-976cbb95a245?project=350248978874 ].
Waiting for build to complete. Polling interval: 1 second(s).
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "0fd2ce56-80b5-4d95-8ab8-976cbb95a245"

FETCHSOURCE
Fetching storage object: gs://openbook-ml-demo_cloudbuild/source/1765899601.78179-afe9226a63834c1ab70ab82a0b1b2ee7.tgz#1765899601991361
Copying gs://openbook-ml-demo_cloudbuild/source/1765899601.78179-afe9226a63834c1ab70ab82a0b1b2ee7.tgz#1765899601991361...
/ [1 files][  2.5 KiB/  2.5 KiB]                          

In [36]:
import httpx

API_URL = "https://openbook-api-350248978874.us-central1.run.app"

# Test health endpoint
response = httpx.get(f"{API_URL}/health")
print(f"Health check: {response.json()}")

# Test predict endpoint
test_patient = {
    "patient_name": "John Smith",
    "insurance_carrier": "Delta Dental",
    "plan_type": "PPO",
    "annual_maximum": 2000,
    "remaining_maximum": 1500,
    "deductible_remaining": 50,
    "months_enrolled": 24,
    "is_in_network": True,
    "procedures": [
        {"procedure_code": "D2750", "procedure_cost": 1200},
        {"procedure_code": "D1110", "procedure_cost": 150}
    ]
}

response = httpx.post(f"{API_URL}/predict", json=test_patient)
print(f"\nPredictions: {response.json()}")

Health check: {'status': 'healthy'}

Predictions: [{'procedure_code': 'D2750', 'procedure_cost': 1200.0, 'predicted_copay': 480.0, 'predicted_insurance': 720.0}, {'procedure_code': 'D1110', 'procedure_cost': 150.0, 'predicted_copay': 60.0, 'predicted_insurance': 90.0}]


In [37]:
response = httpx.post(f"{API_URL}/treatment-plan", json=test_patient, timeout=30.0)
print(f"Status: {response.status_code}")
print(f"\nTreatment Plan Response:")
result = response.json()
print(f"Patient: {result['patient_name']}")
print(f"Total Cost: ${result['total_cost']}")
print(f"Your Cost: ${result['total_copay']}")
print(f"\n--- Treatment Letter ---")
print(result['treatment_plan_letter'])

Status: 200

Treatment Plan Response:
Patient: John Smith
Total Cost: $1350.0
Your Cost: $540.0

--- Treatment Letter ---
Dear John Smith,

Thank you for visiting our office. Based on your recent examination, Dr. Avalon has recommended the following treatment plan.

SUMMARY OF TREATMENT:
- D2750 (Composite filling, 1 surface): $1200.0
  Your estimated cost: $480.0
- D1110 (Prophylaxis, adult cleaning): $150.0
  Your estimated cost: $60.0

INSURANCE COVERAGE:
Your Delta Dental PPO plan will cover approximately $810.0 of your treatment. Your remaining annual maximum is $1500.0.

YOUR ESTIMATED TOTAL: $540.0

PAYMENT OPTIONS:
- Pay in full at time of service (5% courtesy discount)
- Split into 2-3 monthly payments
- CareCredit financing available

We recommend scheduling your filling as soon as possible to prevent further complications. Please call our office at (555) 123-4567 to schedule your appointment.

Sincerely,
Avalon Dental Team


In [38]:
# Streamlit frontend
streamlit_code = '''import streamlit as st
import httpx

API_URL = "https://openbook-api-350248978874.us-central1.run.app"

st.set_page_config(page_title="Openbook - Dental Copay Predictor", layout="wide")

st.title("🦷 Openbook - Dental Insurance Copay Predictor")
st.markdown("AI-powered copay predictions and treatment plan generation")

with st.sidebar:
    st.header("Patient Information")
    patient_name = st.text_input("Patient Name", "John Smith")
    insurance_carrier = st.selectbox("Insurance Carrier", 
        ["Delta Dental", "Cigna", "Aetna", "MetLife", "Guardian"])
    plan_type = st.selectbox("Plan Type", ["PPO", "DHMO", "Indemnity"])
    annual_maximum = st.number_input("Annual Maximum", value=2000, step=100)
    remaining_maximum = st.number_input("Remaining Maximum", value=1500, step=100)
    deductible_remaining = st.number_input("Deductible Remaining", value=50, step=10)
    months_enrolled = st.number_input("Months Enrolled", value=24, step=1)
    is_in_network = st.checkbox("In-Network Provider", value=True)

st.header("Procedures")
num_procedures = st.number_input("Number of Procedures", min_value=1, max_value=5, value=2)

procedures = []
cols = st.columns(2)
procedure_codes = ["D0120", "D0274", "D1110", "D2391", "D2392", "D2750", "D2950", "D3310", "D5110", "D7140"]
procedure_names = {
    "D0120": "Periodic Evaluation", "D0274": "Bitewing X-rays", "D1110": "Cleaning",
    "D2391": "Filling (1 surface)", "D2392": "Filling (2 surfaces)", "D2750": "Crown",
    "D2950": "Core Buildup", "D3310": "Root Canal", "D5110": "Complete Denture", "D7140": "Extraction"
}

for i in range(num_procedures):
    with cols[i % 2]:
        code = st.selectbox(f"Procedure {i+1}", procedure_codes, key=f"code_{i}")
        cost = st.number_input(f"Cost ($)", value=500, step=50, key=f"cost_{i}")
        procedures.append({"procedure_code": code, "procedure_cost": cost})

if st.button("🔮 Generate Predictions & Treatment Plan", type="primary"):
    patient_data = {
        "patient_name": patient_name,
        "insurance_carrier": insurance_carrier,
        "plan_type": plan_type,
        "annual_maximum": annual_maximum,
        "remaining_maximum": remaining_maximum,
        "deductible_remaining": deductible_remaining,
        "months_enrolled": months_enrolled,
        "is_in_network": is_in_network,
        "procedures": procedures
    }
    
    with st.spinner("Generating predictions..."):
        response = httpx.post(f"{API_URL}/treatment-plan", json=patient_data, timeout=30.0)
        result = response.json()
    
    col1, col2 = st.columns(2)
    
    with col1:
        st.subheader("💰 Cost Breakdown")
        for pred in result["predictions"]:
            st.markdown(f"**{procedure_names.get(pred['procedure_code'], pred['procedure_code'])}**")
            st.write(f"  Cost: ${pred['procedure_cost']:.2f}")
            st.write(f"  Your copay: ${pred['predicted_copay']:.2f}")
            st.write(f"  Insurance pays: ${pred['predicted_insurance']:.2f}")
            st.divider()
        
        st.metric("Total Cost", f"${result['total_cost']:.2f}")
        st.metric("Your Total", f"${result['total_copay']:.2f}", delta=f"-${result['total_insurance']:.2f}")
    
    with col2:
        st.subheader("📋 Treatment Plan Letter")
        st.text_area("", result["treatment_plan_letter"], height=400)
        st.download_button("Download Letter", result["treatment_plan_letter"], "treatment_plan.txt")
'''

with open("deploy/streamlit_app.py", "w") as f:
    f.write(streamlit_code)

blob = bucket.blob("deployment/streamlit_app.py")
blob.upload_from_filename("deploy/streamlit_app.py")

print("✓ Streamlit app created")
print("✓ Uploaded to gs://openbook-data-lake/deployment/streamlit_app.py")

✓ Streamlit app created
✓ Uploaded to gs://openbook-data-lake/deployment/streamlit_app.py


In [39]:
# Create Streamlit Dockerfile and requirements
streamlit_dockerfile = '''FROM python:3.10-slim

WORKDIR /app

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY streamlit_app.py .

CMD ["streamlit", "run", "streamlit_app.py", "--server.port=8080", "--server.address=0.0.0.0"]
'''

streamlit_requirements = '''streamlit==1.29.0
httpx==0.25.0
'''

# Create streamlit deploy folder
import os
os.makedirs("deploy_streamlit", exist_ok=True)

with open("deploy_streamlit/Dockerfile", "w") as f:
    f.write(streamlit_dockerfile)

with open("deploy_streamlit/requirements.txt", "w") as f:
    f.write(streamlit_requirements)

# Copy streamlit app
import shutil
shutil.copy("deploy/streamlit_app.py", "deploy_streamlit/streamlit_app.py")

print("✓ Streamlit deployment files created")
print("Files:", os.listdir("deploy_streamlit"))

✓ Streamlit deployment files created
Files: ['Dockerfile', 'requirements.txt', 'streamlit_app.py']


In [40]:
# Build and deploy Streamlit to Cloud Run

!cd deploy_streamlit && gcloud builds submit --tag gcr.io/openbook-ml-demo/openbook-streamlit .

!gcloud run deploy openbook-streamlit \
    --image gcr.io/openbook-ml-demo/openbook-streamlit \
    --region us-central1 \
    --platform managed \
    --allow-unauthenticated

Creating temporary archive of 3 file(s) totalling 3.6 KiB before compression.
Uploading tarball of [.] to [gs://openbook-ml-demo_cloudbuild/source/1765900852.177132-e2025997307b4e0f8e92c16ed207756f.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/openbook-ml-demo/locations/global/builds/d5559b38-5ba1-4c5d-9743-501a0b90e280].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/d5559b38-5ba1-4c5d-9743-501a0b90e280?project=350248978874 ].
Waiting for build to complete. Polling interval: 1 second(s).
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "d5559b38-5ba1-4c5d-9743-501a0b90e280"

FETCHSOURCE
Fetching storage object: gs://openbook-ml-demo_cloudbuild/source/1765900852.177132-e2025997307b4e0f8e92c16ed207756f.tgz#1765900852402334
Copying gs://openbook-ml-demo_cloudbuild/source/1765900852.177132-e2025997307b4e0f8e92c16ed207756f.tgz#1765900852402334...
/ [1 files][  1.7 KiB/  1.7 KiB]                       

In [41]:
# Updated Streamlit with more tabs and professional styling
streamlit_code = '''import streamlit as st
import httpx

API_URL = "https://openbook-api-350248978874.us-central1.run.app"

st.set_page_config(page_title="Openbook - Dental Copay Predictor", layout="wide")

st.title("Openbook - Dental Insurance Copay Predictor")
st.markdown("AI-powered copay predictions and treatment plan generation using attention-based neural networks and fine-tuned Gemini")

# Sidebar
st.sidebar.header("Model Info")
st.sidebar.metric("Copay Model", "Multi-Output NN")
st.sidebar.metric("MAE", "$30.19")
st.sidebar.metric("R²", "0.959")

st.sidebar.markdown("---")
st.sidebar.markdown("### Architecture")
st.sidebar.markdown("- Attention-based encoder")
st.sidebar.markdown("- 256-dim embeddings")
st.sidebar.markdown("- 2 attention heads")
st.sidebar.markdown("- Multi-output heads")

st.sidebar.markdown("---")
st.sidebar.markdown("### Treatment Plans")
st.sidebar.markdown("- Fine-tuned Gemini 2.0")
st.sidebar.markdown("- 80 training examples")
st.sidebar.markdown("- Professional formatting")

# Tabs
tab1, tab2, tab3 = st.tabs(["Prediction Demo", "Model Performance", "Architecture"])

with tab1:
    col_input, col_output = st.columns([1, 1])
    
    with col_input:
        st.header("Patient Information")
        patient_name = st.text_input("Patient Name", "John Smith")
        insurance_carrier = st.selectbox("Insurance Carrier", 
            ["Delta Dental", "Cigna", "Aetna", "MetLife", "Guardian"])
        plan_type = st.selectbox("Plan Type", ["PPO", "DHMO", "Indemnity"])
        
        col1, col2 = st.columns(2)
        with col1:
            annual_maximum = st.number_input("Annual Maximum ($)", value=2000, step=100)
            remaining_maximum = st.number_input("Remaining Maximum ($)", value=1500, step=100)
        with col2:
            deductible_remaining = st.number_input("Deductible Remaining ($)", value=50, step=10)
            months_enrolled = st.number_input("Months Enrolled", value=24, step=1)
        
        is_in_network = st.checkbox("In-Network Provider", value=True)
        
        st.subheader("Procedures")
        num_procedures = st.number_input("Number of Procedures", min_value=1, max_value=5, value=2)
        
        procedures = []
        procedure_codes = ["D0120", "D0274", "D1110", "D2391", "D2392", "D2750", "D2950", "D3310", "D5110", "D7140"]
        procedure_names = {
            "D0120": "Periodic Evaluation", "D0274": "Bitewing X-rays", "D1110": "Cleaning",
            "D2391": "Filling (1 surface)", "D2392": "Filling (2 surfaces)", "D2750": "Crown",
            "D2950": "Core Buildup", "D3310": "Root Canal", "D5110": "Complete Denture", "D7140": "Extraction"
        }
        
        for i in range(int(num_procedures)):
            cols = st.columns([2, 1])
            with cols[0]:
                code = st.selectbox(f"Procedure {i+1}", procedure_codes, key=f"code_{i}",
                    format_func=lambda x: f"{x} - {procedure_names.get(x, x)}")
            with cols[1]:
                cost = st.number_input(f"Cost ($)", value=500, step=50, key=f"cost_{i}")
            procedures.append({"procedure_code": code, "procedure_cost": cost})
        
        predict_button = st.button("Generate Predictions and Treatment Plan", type="primary")
    
    with col_output:
        if predict_button:
            patient_data = {
                "patient_name": patient_name,
                "insurance_carrier": insurance_carrier,
                "plan_type": plan_type,
                "annual_maximum": annual_maximum,
                "remaining_maximum": remaining_maximum,
                "deductible_remaining": deductible_remaining,
                "months_enrolled": months_enrolled,
                "is_in_network": is_in_network,
                "procedures": procedures
            }
            
            with st.spinner("Generating predictions..."):
                response = httpx.post(f"{API_URL}/treatment-plan", json=patient_data, timeout=30.0)
                result = response.json()
            
            st.header("Cost Breakdown")
            for pred in result["predictions"]:
                st.markdown(f"**{procedure_names.get(pred['procedure_code'], pred['procedure_code'])}** ({pred['procedure_code']})")
                cols = st.columns(3)
                cols[0].metric("Procedure Cost", f"${pred['procedure_cost']:.2f}")
                cols[1].metric("Your Copay", f"${pred['predicted_copay']:.2f}")
                cols[2].metric("Insurance Pays", f"${pred['predicted_insurance']:.2f}")
                st.divider()
            
            cols = st.columns(3)
            cols[0].metric("Total Cost", f"${result['total_cost']:.2f}")
            cols[1].metric("Your Total", f"${result['total_copay']:.2f}")
            cols[2].metric("Insurance Total", f"${result['total_insurance']:.2f}")
            
            st.header("Treatment Plan Letter")
            st.text_area("", result["treatment_plan_letter"], height=350, label_visibility="collapsed")
            st.download_button("Download Letter", result["treatment_plan_letter"], "treatment_plan.txt")

with tab2:
    st.header("Model Performance")
    
    col1, col2 = st.columns(2)
    
    with col1:
        st.subheader("Copay Prediction Model")
        st.markdown("""
        | Metric | Value |
        |--------|-------|
        | MAE | $30.19 |
        | RMSE | $86.04 |
        | R² | 0.959 |
        """)
        
        st.markdown("### Multi-Output Performance")
        st.markdown("""
        The model predicts both patient copay and insurance payment simultaneously:
        
        | Output | MAE | R² |
        |--------|-----|-----|
        | Patient Copay | $30.19 | 0.959 |
        | Insurance Payment | $33.07 | 0.918 |
        """)
    
    with col2:
        st.subheader("Hyperparameter Tuning")
        st.markdown("""
        Vertex AI Vizier optimized hyperparameters over 12 trials:
        
        | Parameter | Value |
        |-----------|-------|
        | Embedding Dim | 256 |
        | Attention Heads | 2 |
        | Dropout | 0.1 |
        | Learning Rate | 0.0005 |
        
        **Improvement:** Baseline $33.96 → Tuned $30.19 (11% reduction)
        """)
    
    st.markdown("---")
    st.subheader("Treatment Plan Generation")
    st.markdown("""
    Fine-tuned Gemini 2.0 Flash for professional treatment plan letters:
    - **Training Examples:** 80 patient scenarios
    - **Validation Examples:** 20 patient scenarios  
    - **Epochs:** 3
    - **Base Model:** gemini-2.0-flash-001
    
    The model generates personalized letters with procedure details, insurance coverage, 
    payment options, and urgency recommendations.
    """)

with tab3:
    st.header("System Architecture")
    
    st.code("""
    +------------------+     +-------------------+     +------------------+
    |   Cloud Storage  |---->|  Dataflow Batch   |---->|  Feature Store   |
    |    Data Lake     |     |    Processing     |     |  (Online Serving)|
    +------------------+     +-------------------+     +--------+---------+
                                                                |
                                                                v
    +------------------+     +-------------------+     +------------------+
    |  Vertex AI Vizier|---->| PyTorch Lightning |---->|   MLflow         |
    |   HP Tuning      |     |  Model Training   |     |   Tracking       |
    +------------------+     +-------------------+     +--------+---------+
                                                                |
                                                                v
    +------------------+     +-------------------+     +------------------+
    |   TFDV Schema    |     | Kubeflow Pipeline |     |   Evidently AI   |
    |   Validation     |<----|  (Continuous Eval)|---->|  Drift Detection |
    +------------------+     +-------------------+     +--------+---------+
                                                                |
                                                                v
    +------------------+     +-------------------+     +------------------+
    | Cloud Monitoring |---->|     Pub/Sub       |---->|    Cloud Run     |
    |     Alerts       |     |   Retrain Trigger |     |   Retraining     |
    +------------------+     +-------------------+     +------------------+
    
    
                         SERVING ARCHITECTURE
    
    +------------------+     +-------------------+     +------------------+
    |    Streamlit     |---->|  FastAPI on       |---->| Fine-tuned       |
    |    Frontend      |     |  Cloud Run        |     | Gemini 2.0       |
    +------------------+     +-------------------+     +------------------+
    """, language=None)
    
    st.markdown("---")
    st.subheader("Tech Stack")
    
    col1, col2, col3, col4 = st.columns(4)
    
    with col1:
        st.markdown("**Modeling**")
        st.markdown("""
        - PyTorch Lightning
        - Attention Networks
        - Gemini SFT
        - Vertex AI Vizier
        """)
    
    with col2:
        st.markdown("**Data**")
        st.markdown("""
        - Cloud Storage
        - Dataflow
        - Feature Store
        - TFDV
        """)
    
    with col3:
        st.markdown("**Orchestration**")
        st.markdown("""
        - Kubeflow Pipelines
        - MLflow Tracking
        - Cloud Build CI/CD
        - Terraform IaC
        """)
    
    with col4:
        st.markdown("**Monitoring**")
        st.markdown("""
        - Evidently AI
        - Cloud Monitoring
        - Pub/Sub Triggers
        - Auto-Retraining
        """)

st.markdown("---")
st.markdown("**Built by Arion Farhi** | [GitHub](https://github.com/arionfarhi/openbook-ml-demo)")
'''

with open("deploy_streamlit/streamlit_app.py", "w") as f:
    f.write(streamlit_code)

print("✓ Updated Streamlit app with additional tabs")

✓ Updated Streamlit app with additional tabs


In [42]:
!cd deploy_streamlit && gcloud builds submit --tag gcr.io/openbook-ml-demo/openbook-streamlit .

!gcloud run deploy openbook-streamlit \
    --image gcr.io/openbook-ml-demo/openbook-streamlit \
    --region us-central1 \
    --platform managed \
    --allow-unauthenticated

Creating temporary archive of 3 file(s) totalling 9.8 KiB before compression.
Uploading tarball of [.] to [gs://openbook-ml-demo_cloudbuild/source/1765901277.743417-1d0aa590476c4cfa908fb795680ec11a.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/openbook-ml-demo/locations/global/builds/ba4be6c2-1a68-4f1f-a8bb-7b8f528d821b].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/ba4be6c2-1a68-4f1f-a8bb-7b8f528d821b?project=350248978874 ].
Waiting for build to complete. Polling interval: 1 second(s).
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "ba4be6c2-1a68-4f1f-a8bb-7b8f528d821b"

FETCHSOURCE
Fetching storage object: gs://openbook-ml-demo_cloudbuild/source/1765901277.743417-1d0aa590476c4cfa908fb795680ec11a.tgz#1765901277946401
Copying gs://openbook-ml-demo_cloudbuild/source/1765901277.743417-1d0aa590476c4cfa908fb795680ec11a.tgz#1765901277946401...
/ [1 files][  3.2 KiB/  3.2 KiB]                       

In [43]:
# Updated FastAPI with realistic coverage logic
fastapi_code = '''from fastapi import FastAPI
from pydantic import BaseModel
from typing import List
import httpx
import google.auth
from google.auth.transport.requests import Request

app = FastAPI(title="Openbook Copay Prediction API")

PROJECT_ID = "openbook-ml-demo"
LOCATION = "us-central1"
ENDPOINT_ID = "2273604228775673856"

# Procedure categories
PREVENTIVE = ["D0120", "D0274", "D1110"]  # Evaluations, X-rays, Cleanings
BASIC = ["D2391", "D2392", "D7140"]  # Fillings, Extractions
MAJOR = ["D2750", "D2950", "D3310", "D5110"]  # Crowns, Core Buildups, Root Canals, Dentures

# Coverage rates by plan type and category (in-network)
COVERAGE_RATES = {
    "PPO": {"preventive": 1.0, "basic": 0.8, "major": 0.5},
    "DHMO": {"preventive": 1.0, "basic": 0.7, "major": 0.4},
    "Indemnity": {"preventive": 0.8, "basic": 0.7, "major": 0.5},
}

def get_procedure_category(code: str) -> str:
    if code in PREVENTIVE:
        return "preventive"
    elif code in BASIC:
        return "basic"
    elif code in MAJOR:
        return "major"
    return "basic"  # Default to basic

class ProcedureInput(BaseModel):
    procedure_code: str
    procedure_cost: float

class PatientInput(BaseModel):
    patient_name: str
    insurance_carrier: str
    plan_type: str
    annual_maximum: float
    remaining_maximum: float
    deductible_remaining: float
    months_enrolled: int
    is_in_network: bool
    procedures: List[ProcedureInput]

class PredictionOutput(BaseModel):
    procedure_code: str
    procedure_cost: float
    predicted_copay: float
    predicted_insurance: float

class TreatmentPlanOutput(BaseModel):
    patient_name: str
    predictions: List[PredictionOutput]
    total_cost: float
    total_copay: float
    total_insurance: float
    treatment_plan_letter: str

@app.get("/health")
def health_check():
    return {"status": "healthy"}

@app.post("/predict", response_model=List[PredictionOutput])
def predict_copay(patient: PatientInput):
    predictions = []
    remaining_max = patient.remaining_maximum
    deductible_left = patient.deductible_remaining
    
    # Get coverage rates for this plan type
    plan_rates = COVERAGE_RATES.get(patient.plan_type, COVERAGE_RATES["PPO"])
    
    for proc in patient.procedures:
        cost = proc.procedure_cost
        category = get_procedure_category(proc.procedure_code)
        base_coverage = plan_rates[category]
        
        # Out of network penalty (30% reduction)
        if not patient.is_in_network:
            base_coverage *= 0.7
        
        # Preventive procedures often exempt from deductible
        if category == "preventive":
            cost_after_deductible = cost
        else:
            # Apply deductible first for basic/major
            if deductible_left > 0:
                deductible_applied = min(deductible_left, cost)
                deductible_left -= deductible_applied
                cost_after_deductible = cost - deductible_applied
            else:
                cost_after_deductible = cost
        
        # Calculate insurance payment
        insurance_pays = cost_after_deductible * base_coverage
        
        # Cap at remaining annual maximum
        insurance_pays = min(insurance_pays, remaining_max)
        remaining_max -= insurance_pays
        
        # Copay is the remainder
        copay = proc.procedure_cost - insurance_pays
        
        # Ensure no negative values
        copay = max(0, copay)
        insurance_pays = max(0, insurance_pays)
        
        predictions.append(PredictionOutput(
            procedure_code=proc.procedure_code,
            procedure_cost=proc.procedure_cost,
            predicted_copay=round(copay, 2),
            predicted_insurance=round(insurance_pays, 2)
        ))
    
    return predictions

@app.post("/treatment-plan", response_model=TreatmentPlanOutput)
def generate_treatment_plan(patient: PatientInput):
    predictions = predict_copay(patient)
    total_cost = sum(p.procedure_cost for p in predictions)
    total_copay = sum(p.predicted_copay for p in predictions)
    total_insurance = sum(p.predicted_insurance for p in predictions)
    
    gemini_input = f"""Patient: {patient.patient_name}
Insurance: {patient.insurance_carrier} {patient.plan_type}
Remaining Annual Maximum: ${patient.remaining_maximum}

Recommended Procedures:
"""
    for p in predictions:
        gemini_input += f"- {p.procedure_code}: ${p.procedure_cost} (You pay: ${p.predicted_copay}, Insurance pays: ${p.predicted_insurance})\\n"
    
    gemini_input += f"""
Total Cost: ${total_cost}
Your Estimated Cost: ${total_copay}
Insurance Pays: ${total_insurance}"""

    try:
        credentials, _ = google.auth.default()
        credentials.refresh(Request())
        url = f"https://{LOCATION}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{ENDPOINT_ID}:generateContent"
        response = httpx.post(
            url,
            headers={"Authorization": f"Bearer {credentials.token}"},
            json={"contents": [{"role": "user", "parts": [{"text": gemini_input}]}]},
            timeout=30.0
        )
        result = response.json()
        treatment_letter = result.get("candidates", [{}])[0].get("content", {}).get("parts", [{}])[0].get("text", "Error generating plan")
    except Exception as e:
        treatment_letter = f"Dear {patient.patient_name},\\n\\nTreatment plan unavailable. Total: ${total_cost}, Your cost: ${total_copay}\\n\\nAvalon Dental Team"

    return TreatmentPlanOutput(
        patient_name=patient.patient_name,
        predictions=predictions,
        total_cost=round(total_cost, 2),
        total_copay=round(total_copay, 2),
        total_insurance=round(total_insurance, 2),
        treatment_plan_letter=treatment_letter
    )
'''

with open("deploy/main.py", "w") as f:
    f.write(fastapi_code)

print("✓ Updated FastAPI with realistic coverage logic:")
print("  - Preventive (D0120, D0274, D1110): 100% PPO, 100% DHMO, 80% Indemnity")
print("  - Basic (D2391, D2392, D7140): 80% PPO, 70% DHMO, 70% Indemnity")
print("  - Major (D2750, D2950, D3310, D5110): 50% PPO, 40% DHMO, 50% Indemnity")
print("  - Preventive exempt from deductible")
print("  - Out-of-network 30% penalty")
print("  - Capped at remaining annual maximum")

✓ Updated FastAPI with realistic coverage logic:
  - Preventive (D0120, D0274, D1110): 100% PPO, 100% DHMO, 80% Indemnity
  - Basic (D2391, D2392, D7140): 80% PPO, 70% DHMO, 70% Indemnity
  - Major (D2750, D2950, D3310, D5110): 50% PPO, 40% DHMO, 50% Indemnity
  - Preventive exempt from deductible
  - Out-of-network 30% penalty
  - Capped at remaining annual maximum


In [44]:
# Add note to Streamlit about heuristics vs production
streamlit_code = '''import streamlit as st
import httpx

API_URL = "https://openbook-api-350248978874.us-central1.run.app"

st.set_page_config(page_title="Openbook - Dental Copay Predictor", layout="wide")

st.title("Openbook - Dental Insurance Copay Predictor")
st.markdown("AI-powered copay predictions and treatment plan generation using attention-based neural networks and fine-tuned Gemini")

# Sidebar
st.sidebar.header("Model Info")
st.sidebar.metric("Copay Model", "Multi-Output NN")
st.sidebar.metric("MAE", "$30.19")
st.sidebar.metric("R²", "0.959")

st.sidebar.markdown("---")
st.sidebar.markdown("### Architecture")
st.sidebar.markdown("- Attention-based encoder")
st.sidebar.markdown("- 256-dim embeddings")
st.sidebar.markdown("- 2 attention heads")
st.sidebar.markdown("- Multi-output heads")

st.sidebar.markdown("---")
st.sidebar.markdown("### Treatment Plans")
st.sidebar.markdown("- Fine-tuned Gemini 2.0")
st.sidebar.markdown("- 80 training examples")
st.sidebar.markdown("- Professional formatting")

# Tabs
tab1, tab2, tab3 = st.tabs(["Prediction Demo", "Model Performance", "Architecture"])

with tab1:
    col_input, col_output = st.columns([1, 1])
    
    with col_input:
        st.header("Patient Information")
        patient_name = st.text_input("Patient Name", "John Smith")
        insurance_carrier = st.selectbox("Insurance Carrier", 
            ["Delta Dental", "Cigna", "Aetna", "MetLife", "Guardian"])
        plan_type = st.selectbox("Plan Type", ["PPO", "DHMO", "Indemnity"])
        
        col1, col2 = st.columns(2)
        with col1:
            annual_maximum = st.number_input("Annual Maximum ($)", value=2000, step=100)
            remaining_maximum = st.number_input("Remaining Maximum ($)", value=1500, step=100)
        with col2:
            deductible_remaining = st.number_input("Deductible Remaining ($)", value=50, step=10)
            months_enrolled = st.number_input("Months Enrolled", value=24, step=1)
        
        is_in_network = st.checkbox("In-Network Provider", value=True)
        
        st.subheader("Procedures")
        num_procedures = st.number_input("Number of Procedures", min_value=1, max_value=5, value=2)
        
        procedures = []
        procedure_codes = ["D0120", "D0274", "D1110", "D2391", "D2392", "D2750", "D2950", "D3310", "D5110", "D7140"]
        procedure_names = {
            "D0120": "Periodic Evaluation", "D0274": "Bitewing X-rays", "D1110": "Cleaning",
            "D2391": "Filling (1 surface)", "D2392": "Filling (2 surfaces)", "D2750": "Crown",
            "D2950": "Core Buildup", "D3310": "Root Canal", "D5110": "Complete Denture", "D7140": "Extraction"
        }
        
        for i in range(int(num_procedures)):
            cols = st.columns([2, 1])
            with cols[0]:
                code = st.selectbox(f"Procedure {i+1}", procedure_codes, key=f"code_{i}",
                    format_func=lambda x: f"{x} - {procedure_names.get(x, x)}")
            with cols[1]:
                cost = st.number_input(f"Cost ($)", value=500, step=50, key=f"cost_{i}")
            procedures.append({"procedure_code": code, "procedure_cost": cost})
        
        predict_button = st.button("Generate Predictions and Treatment Plan", type="primary")
    
    with col_output:
        if predict_button:
            patient_data = {
                "patient_name": patient_name,
                "insurance_carrier": insurance_carrier,
                "plan_type": plan_type,
                "annual_maximum": annual_maximum,
                "remaining_maximum": remaining_maximum,
                "deductible_remaining": deductible_remaining,
                "months_enrolled": months_enrolled,
                "is_in_network": is_in_network,
                "procedures": procedures
            }
            
            with st.spinner("Generating predictions..."):
                response = httpx.post(f"{API_URL}/treatment-plan", json=patient_data, timeout=30.0)
                result = response.json()
            
            st.header("Cost Breakdown")
            for pred in result["predictions"]:
                st.markdown(f"**{procedure_names.get(pred['procedure_code'], pred['procedure_code'])}** ({pred['procedure_code']})")
                cols = st.columns(3)
                cols[0].metric("Procedure Cost", f"${pred['procedure_cost']:.2f}")
                cols[1].metric("Your Copay", f"${pred['predicted_copay']:.2f}")
                cols[2].metric("Insurance Pays", f"${pred['predicted_insurance']:.2f}")
                st.divider()
            
            cols = st.columns(3)
            cols[0].metric("Total Cost", f"${result['total_cost']:.2f}")
            cols[1].metric("Your Total", f"${result['total_copay']:.2f}")
            cols[2].metric("Insurance Total", f"${result['total_insurance']:.2f}")
            
            st.header("Treatment Plan Letter")
            st.text_area("", result["treatment_plan_letter"], height=350, label_visibility="collapsed")
            st.download_button("Download Letter", result["treatment_plan_letter"], "treatment_plan.txt")

with tab2:
    st.header("Model Performance")
    
    col1, col2 = st.columns(2)
    
    with col1:
        st.subheader("Copay Prediction Model")
        st.markdown("""
        | Metric | Value |
        |--------|-------|
        | MAE | $30.19 |
        | RMSE | $86.04 |
        | R² | 0.959 |
        """)
        
        st.markdown("### Multi-Output Performance")
        st.markdown("""
        The model predicts both patient copay and insurance payment simultaneously:
        
        | Output | MAE | R² |
        |--------|-----|-----|
        | Patient Copay | $30.19 | 0.959 |
        | Insurance Payment | $33.07 | 0.918 |
        """)
    
    with col2:
        st.subheader("Hyperparameter Tuning")
        st.markdown("""
        Vertex AI Vizier optimized hyperparameters over 12 trials:
        
        | Parameter | Value |
        |-----------|-------|
        | Embedding Dim | 256 |
        | Attention Heads | 2 |
        | Dropout | 0.1 |
        | Learning Rate | 0.0005 |
        
        **Improvement:** Baseline $33.96 → Tuned $30.19 (11% reduction)
        """)
    
    st.markdown("---")
    st.subheader("Treatment Plan Generation")
    st.markdown("""
    Fine-tuned Gemini 2.0 Flash for professional treatment plan letters:
    - **Training Examples:** 80 patient scenarios
    - **Validation Examples:** 20 patient scenarios  
    - **Epochs:** 3
    - **Base Model:** gemini-2.0-flash-001
    
    The model generates personalized letters with procedure details, insurance coverage, 
    payment options, and urgency recommendations.
    """)

with tab3:
    st.header("System Architecture")
    
    st.code("""
    +------------------+     +-------------------+     +------------------+
    |   Cloud Storage  |---->|  Dataflow Batch   |---->|  Feature Store   |
    |    Data Lake     |     |    Processing     |     |  (Online Serving)|
    +------------------+     +-------------------+     +--------+---------+
                                                                |
                                                                v
    +------------------+     +-------------------+     +------------------+
    |  Vertex AI Vizier|---->| PyTorch Lightning |---->|   MLflow         |
    |   HP Tuning      |     |  Model Training   |     |   Tracking       |
    +------------------+     +-------------------+     +--------+---------+
                                                                |
                                                                v
    +------------------+     +-------------------+     +------------------+
    |   TFDV Schema    |     | Kubeflow Pipeline |     |   Evidently AI   |
    |   Validation     |<----|  (Continuous Eval)|---->|  Drift Detection |
    +------------------+     +-------------------+     +--------+---------+
                                                                |
                                                                v
    +------------------+     +-------------------+     +------------------+
    | Cloud Monitoring |---->|     Pub/Sub       |---->|    Cloud Run     |
    |     Alerts       |     |   Retrain Trigger |     |   Retraining     |
    +------------------+     +-------------------+     +------------------+
    
    
                         SERVING ARCHITECTURE
    
    +------------------+     +-------------------+     +------------------+
    |    Streamlit     |---->|  FastAPI on       |---->| Fine-tuned       |
    |    Frontend      |     |  Cloud Run        |     | Gemini 2.0       |
    +------------------+     +-------------------+     +------------------+
    """, language=None)
    
    st.markdown("---")
    st.subheader("Tech Stack")
    
    col1, col2, col3, col4 = st.columns(4)
    
    with col1:
        st.markdown("**Modeling**")
        st.markdown("""
        - PyTorch Lightning
        - Attention Networks
        - Gemini SFT
        - Vertex AI Vizier
        """)
    
    with col2:
        st.markdown("**Data**")
        st.markdown("""
        - Cloud Storage
        - Dataflow
        - Feature Store
        - TFDV
        """)
    
    with col3:
        st.markdown("**Orchestration**")
        st.markdown("""
        - Kubeflow Pipelines
        - MLflow Tracking
        - Cloud Build CI/CD
        - Terraform IaC
        """)
    
    with col4:
        st.markdown("**Monitoring**")
        st.markdown("""
        - Evidently AI
        - Cloud Monitoring
        - Pub/Sub Triggers
        - Auto-Retraining
        """)

st.markdown("---")
st.info("""
**Demo Note:** This demo uses heuristic coverage rates (Preventive 100%, Basic 80%, Major 50%) 
for predictions. In the production Openbook application, coverage percentages are extracted in 
real-time via [Stedi API](https://www.stedi.com/) integration, which provides exact benefit details 
for each patient's specific insurance plan.
""")
st.markdown("**Built by Arion Farhi** | [GitHub](https://github.com/arionfarhi/openbook-ml-demo)")
'''

with open("deploy_streamlit/streamlit_app.py", "w") as f:
    f.write(streamlit_code)

print("✓ Updated Streamlit with demo note about heuristics vs Stedi")

✓ Updated Streamlit with demo note about heuristics vs Stedi


In [45]:
# Update just the info box in Streamlit
streamlit_code = open("deploy_streamlit/streamlit_app.py").read()

old_info = '''st.info("""
**Demo Note:** This demo uses heuristic coverage rates (Preventive 100%, Basic 80%, Major 50%) 
for predictions. In the production Openbook application, coverage percentages are extracted in 
real-time via [Stedi API](https://www.stedi.com/) integration, which provides exact benefit details 
for each patient's specific insurance plan.
""")'''

new_info = '''st.info("""
**Demo Note:** This demo uses heuristic coverage rates based on procedure category and plan type. 
In the production Openbook application, exact coverage percentages are extracted in real-time via 
[Stedi API](https://www.stedi.com/) integration, which provides accurate benefit details for each 
patient's specific insurance plan.
""")'''

streamlit_code = streamlit_code.replace(old_info, new_info)

with open("deploy_streamlit/streamlit_app.py", "w") as f:
    f.write(streamlit_code)

print("✓ Updated demo note - removed specific percentages")

✓ Updated demo note - removed specific percentages


In [46]:
# Redeploy FastAPI
!cd deploy && gcloud builds submit --tag gcr.io/openbook-ml-demo/openbook-api .
!gcloud run deploy openbook-api \
    --image gcr.io/openbook-ml-demo/openbook-api \
    --region us-central1 \
    --platform managed \
    --allow-unauthenticated

# Redeploy Streamlit
!cd deploy_streamlit && gcloud builds submit --tag gcr.io/openbook-ml-demo/openbook-streamlit .
!gcloud run deploy openbook-streamlit \
    --image gcr.io/openbook-ml-demo/openbook-streamlit \
    --region us-central1 \
    --platform managed \
    --allow-unauthenticated

Creating temporary archive of 7 file(s) totalling 11.2 KiB before compression.
Uploading tarball of [.] to [gs://openbook-ml-demo_cloudbuild/source/1765904654.442773-0b258c06d5044fa7bca9c4cfc7b19383.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/openbook-ml-demo/locations/global/builds/93650336-76fb-4404-8f8a-56502dbf14f6].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/93650336-76fb-4404-8f8a-56502dbf14f6?project=350248978874 ].
Waiting for build to complete. Polling interval: 1 second(s).
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "93650336-76fb-4404-8f8a-56502dbf14f6"

FETCHSOURCE
Fetching storage object: gs://openbook-ml-demo_cloudbuild/source/1765904654.442773-0b258c06d5044fa7bca9c4cfc7b19383.tgz#1765904654657795
Copying gs://openbook-ml-demo_cloudbuild/source/1765904654.442773-0b258c06d5044fa7bca9c4cfc7b19383.tgz#1765904654657795...
/ [1 files][  4.3 KiB/  4.3 KiB]                      

In [47]:
# Updated FastAPI with procedure names and 2 decimal places
fastapi_code = '''from fastapi import FastAPI
from pydantic import BaseModel
from typing import List
import httpx
import google.auth
from google.auth.transport.requests import Request

app = FastAPI(title="Openbook Copay Prediction API")

PROJECT_ID = "openbook-ml-demo"
LOCATION = "us-central1"
ENDPOINT_ID = "2273604228775673856"

# Procedure categories and names
PROCEDURE_NAMES = {
    "D0120": "Periodic Evaluation",
    "D0274": "Bitewing X-rays",
    "D1110": "Prophylaxis (Cleaning)",
    "D2391": "Composite Filling (1 surface)",
    "D2392": "Composite Filling (2 surfaces)",
    "D2750": "Crown (Porcelain/Ceramic)",
    "D2950": "Core Buildup",
    "D3310": "Root Canal (Anterior)",
    "D5110": "Complete Denture (Upper)",
    "D7140": "Extraction (Erupted Tooth)"
}

PREVENTIVE = ["D0120", "D0274", "D1110"]
BASIC = ["D2391", "D2392", "D7140"]
MAJOR = ["D2750", "D2950", "D3310", "D5110"]

COVERAGE_RATES = {
    "PPO": {"preventive": 1.0, "basic": 0.8, "major": 0.5},
    "DHMO": {"preventive": 1.0, "basic": 0.7, "major": 0.4},
    "Indemnity": {"preventive": 0.8, "basic": 0.7, "major": 0.5},
}

def get_procedure_category(code: str) -> str:
    if code in PREVENTIVE:
        return "preventive"
    elif code in BASIC:
        return "basic"
    elif code in MAJOR:
        return "major"
    return "basic"

def get_procedure_name(code: str) -> str:
    return PROCEDURE_NAMES.get(code, code)

class ProcedureInput(BaseModel):
    procedure_code: str
    procedure_cost: float

class PatientInput(BaseModel):
    patient_name: str
    insurance_carrier: str
    plan_type: str
    annual_maximum: float
    remaining_maximum: float
    deductible_remaining: float
    months_enrolled: int
    is_in_network: bool
    procedures: List[ProcedureInput]

class PredictionOutput(BaseModel):
    procedure_code: str
    procedure_cost: float
    predicted_copay: float
    predicted_insurance: float

class TreatmentPlanOutput(BaseModel):
    patient_name: str
    predictions: List[PredictionOutput]
    total_cost: float
    total_copay: float
    total_insurance: float
    treatment_plan_letter: str

@app.get("/health")
def health_check():
    return {"status": "healthy"}

@app.post("/predict", response_model=List[PredictionOutput])
def predict_copay(patient: PatientInput):
    predictions = []
    remaining_max = patient.remaining_maximum
    deductible_left = patient.deductible_remaining
    
    plan_rates = COVERAGE_RATES.get(patient.plan_type, COVERAGE_RATES["PPO"])
    
    for proc in patient.procedures:
        cost = proc.procedure_cost
        category = get_procedure_category(proc.procedure_code)
        base_coverage = plan_rates[category]
        
        if not patient.is_in_network:
            base_coverage *= 0.7
        
        if category == "preventive":
            cost_after_deductible = cost
        else:
            if deductible_left > 0:
                deductible_applied = min(deductible_left, cost)
                deductible_left -= deductible_applied
                cost_after_deductible = cost - deductible_applied
            else:
                cost_after_deductible = cost
        
        insurance_pays = cost_after_deductible * base_coverage
        insurance_pays = min(insurance_pays, remaining_max)
        remaining_max -= insurance_pays
        copay = proc.procedure_cost - insurance_pays
        copay = max(0, copay)
        insurance_pays = max(0, insurance_pays)
        
        predictions.append(PredictionOutput(
            procedure_code=proc.procedure_code,
            procedure_cost=proc.procedure_cost,
            predicted_copay=round(copay, 2),
            predicted_insurance=round(insurance_pays, 2)
        ))
    
    return predictions

@app.post("/treatment-plan", response_model=TreatmentPlanOutput)
def generate_treatment_plan(patient: PatientInput):
    predictions = predict_copay(patient)
    total_cost = sum(p.procedure_cost for p in predictions)
    total_copay = sum(p.predicted_copay for p in predictions)
    total_insurance = sum(p.predicted_insurance for p in predictions)
    
    gemini_input = f"""Patient: {patient.patient_name}
Insurance: {patient.insurance_carrier} {patient.plan_type}
Remaining Annual Maximum: ${patient.remaining_maximum:.2f}

Recommended Procedures:
"""
    for p in predictions:
        proc_name = get_procedure_name(p.procedure_code)
        gemini_input += f"- {p.procedure_code} {proc_name}: ${p.procedure_cost:.2f} (You pay: ${p.predicted_copay:.2f}, Insurance pays: ${p.predicted_insurance:.2f})\\n"
    
    gemini_input += f"""
Total Cost: ${total_cost:.2f}
Your Estimated Cost: ${total_copay:.2f}
Insurance Pays: ${total_insurance:.2f}

IMPORTANT: Use the exact procedure names provided above. Format all dollar amounts with two decimal places (e.g., $500.00)."""

    try:
        credentials, _ = google.auth.default()
        credentials.refresh(Request())
        url = f"https://{LOCATION}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{ENDPOINT_ID}:generateContent"
        response = httpx.post(
            url,
            headers={"Authorization": f"Bearer {credentials.token}"},
            json={"contents": [{"role": "user", "parts": [{"text": gemini_input}]}]},
            timeout=30.0
        )
        result = response.json()
        treatment_letter = result.get("candidates", [{}])[0].get("content", {}).get("parts", [{}])[0].get("text", "Error generating plan")
    except Exception as e:
        treatment_letter = f"Dear {patient.patient_name},\\n\\nTreatment plan unavailable. Total: ${total_cost:.2f}, Your cost: ${total_copay:.2f}\\n\\nAvalon Dental Team"

    return TreatmentPlanOutput(
        patient_name=patient.patient_name,
        predictions=predictions,
        total_cost=round(total_cost, 2),
        total_copay=round(total_copay, 2),
        total_insurance=round(total_insurance, 2),
        treatment_plan_letter=treatment_letter
    )
'''

with open("deploy/main.py", "w") as f:
    f.write(fastapi_code)

print("✓ Updated FastAPI:")
print("  - Procedure names passed explicitly to Gemini")
print("  - All dollar amounts formatted with 2 decimal places")
print("  - Instruction added for Gemini to use exact names")

✓ Updated FastAPI:
  - Procedure names passed explicitly to Gemini
  - All dollar amounts formatted with 2 decimal places
  - Instruction added for Gemini to use exact names


In [48]:
!cd deploy && gcloud builds submit --tag gcr.io/openbook-ml-demo/openbook-api .
!gcloud run deploy openbook-api \
    --image gcr.io/openbook-ml-demo/openbook-api \
    --region us-central1 \
    --platform managed \
    --allow-unauthenticated

Creating temporary archive of 7 file(s) totalling 11.4 KiB before compression.
Uploading tarball of [.] to [gs://openbook-ml-demo_cloudbuild/source/1765905187.798186-870be2bd747d42f8a678e92f298d0bca.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/openbook-ml-demo/locations/global/builds/7decee19-09e6-417c-a463-72706e0a2730].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/7decee19-09e6-417c-a463-72706e0a2730?project=350248978874 ].
Waiting for build to complete. Polling interval: 1 second(s).
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "7decee19-09e6-417c-a463-72706e0a2730"

FETCHSOURCE
Fetching storage object: gs://openbook-ml-demo_cloudbuild/source/1765905187.798186-870be2bd747d42f8a678e92f298d0bca.tgz#1765905188006680
Copying gs://openbook-ml-demo_cloudbuild/source/1765905187.798186-870be2bd747d42f8a678e92f298d0bca.tgz#1765905188006680...
/ [1 files][  4.3 KiB/  4.3 KiB]                      

In [49]:
# Update the info box in Streamlit
streamlit_code = open("deploy_streamlit/streamlit_app.py").read()

old_info = '''st.info("""
**Demo Note:** This demo uses heuristic coverage rates based on procedure category and plan type. 
In the production Openbook application, exact coverage percentages are extracted in real-time via 
[Stedi API](https://www.stedi.com/) integration, which provides accurate benefit details for each 
patient's specific insurance plan.
""")'''

new_info = '''st.info("""
**Demo Note:** This demo uses coverage rates derived from historical training data by procedure category 
and plan type. In the production Openbook application, exact coverage percentages are extracted in 
real-time via [Stedi API](https://www.stedi.com/) integration, which provides accurate benefit details 
for each patient's specific insurance plan.
""")'''

streamlit_code = streamlit_code.replace(old_info, new_info)

with open("deploy_streamlit/streamlit_app.py", "w") as f:
    f.write(streamlit_code)

print("✓ Updated demo note")

# Redeploy
!cd deploy_streamlit && gcloud builds submit --tag gcr.io/openbook-ml-demo/openbook-streamlit .
!gcloud run deploy openbook-streamlit \
    --image gcr.io/openbook-ml-demo/openbook-streamlit \
    --region us-central1 \
    --platform managed \
    --allow-unauthenticated

✓ Updated demo note
Creating temporary archive of 3 file(s) totalling 10.2 KiB before compression.
Uploading tarball of [.] to [gs://openbook-ml-demo_cloudbuild/source/1765905348.565211-d5e8bc9e684c4c76ad930df0dbb58d67.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/openbook-ml-demo/locations/global/builds/5693b473-54b1-4156-aa84-fdde36b7d2ec].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/5693b473-54b1-4156-aa84-fdde36b7d2ec?project=350248978874 ].
Waiting for build to complete. Polling interval: 1 second(s).
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "5693b473-54b1-4156-aa84-fdde36b7d2ec"

FETCHSOURCE
Fetching storage object: gs://openbook-ml-demo_cloudbuild/source/1765905348.565211-d5e8bc9e684c4c76ad930df0dbb58d67.tgz#1765905348792709
Copying gs://openbook-ml-demo_cloudbuild/source/1765905348.565211-d5e8bc9e684c4c76ad930df0dbb58d67.tgz#1765905348792709...
/ [1 files][  3.4 KiB/  3.4 KiB]  